### Copyright-protected material, all rights reserved. (c) University of Vienna.
_Copyright Notice of the corresponding course at Moodle applies. <br> Only to be used in the MRE course._

# MRE Assignment 2 - Digital Audio Processing 

In this assignment you will load, decode, and process digital audio files (e.g., MP3, WAV) using Python. For the following tasks, you will use our suggested libraries (see the setup section). For both audio formats you will extract and process content and some basic metadata. For the following tasks, you will use our suggested libraries (see the setup section). 

In this notebook, you will implement your solution. This notebook will be imported into the "*_def.ipynb" notebook.

Of course you can include code for testing your implementation in this implementation notebook, but code for testing and output generated for testing is not going to be assessed.

Of course, your code for the solutions in this notebook will be inspected and is subject to grading.

## Setup

For general installation instructions, please refer to the ressources given for all the assignments in Moodle.

If the cell below executes without error, you can start the assignment!

In [2]:
# -------- Imports --------
# Please do not change the contents of this cell!

# Imports required by us.
from enum import Enum
import mutagen      # mutagen
from mutagen.mp3 import MP3
from mutagen.id3 import ID3
from mutagen.easyid3 import EasyID3
import wave         # python's built-in wave library
import pandas as pd # pandas
import ffmpeg       # ffmpeg-python wrapper (requires ffmpeg.exe in your system path!)
import subprocess   # for calling local executables such as ffmpeg.exe


In the cells below, place your own imports, global variables, (helper) functions and classes. Feel free to add cells here as you see fit.

In [355]:
# Please place your own imports here.
import numpy as np
import glob

In [ ]:
# Place any helper functions, global variables and classes here.

## Task 2.1 Organize Audio files by specific criteria (35P):

In [376]:
# Write your function here.

# Auto-plays an audio file and also embeds an IPython audio display.
def MyAudioFilesOrganizer(inputDir: str, grouping) -> pd.DataFrame:
    columns = ['artist',
                'album',
                'genre',
                'format',
                'duration',
                'title',
                'date',
                'sample rate',
                'bitrate',
                'track',
                'composer',
                'encoder', 'channels']
  
    directory = glob.glob(inputDir)
    rows = []
    df = pd.DataFrame()

    for file in directory:
        with open(file, 'rb') as fl:
            audioFile = mutagen.File(fl, easy = True)
            
            if fl.name.endswith(".mp3") :
                fileformat = 'mp3'
                audioFile = mutagen.File(fl, easy = False)
                mf = mutagen.mp3.MP3(audioFile)
                print(mfmutagen.mp3.MPEGInfo(mf))
            elif fl.name.endswith('.wav') :
                fileformat = 'wav'
            elif fl.name.endswith('.flac'):
                fileformat = 'flac'

            if(audioFile.tags is None):
                waveFile = wave.open(fl.name, mode=None)
                #rows = np.append(rows, fl.name)
                bitrate = waveFile.getnframes()
                channels = waveFile.getnchannels()
                samplerate = waveFile.getframerate()
                print(bitrate/float(samplerate))
                print(bitrate)
                #data = [artist, album, genre, fileformat, duration, title, date, samplerate, bitrate, track, composer, encoder, channels]
                #df2 = pd.DataFrame(data = [data],columns = columns)
                #df = df.append(df2 ,ignore_index = True) 
            else:
                rows = np.append(rows, fl.name)
                title = audioFile['title']
                artist = audioFile['artist']
                album = audioFile['album']
                genre = audioFile['genre']
                #fileformat = mutagen.File(fl, easy = False)
                duration = audioFile['title']
                date = audioFile['date']
                samplerate = audioFile['title']
                bitrate = audioFile['title']
                track = audioFile['tracknumber']
                composer = audioFile['composer']
                encoder = audioFile['encodedby']
                channels = 4
                                                                    
                data = [artist, album, genre, fileformat, duration, title, date, samplerate, bitrate, track, composer, encoder, channels]
                df2 = pd.DataFrame(data = [data],columns = columns)
                df = df.append(df2 ,ignore_index = True) 
            
    
    df.index = rows
    
    #df['artist'] = df['artist'].apply(tuple)
    
    
    
    #group = df.groupby('artist')
        
    #return group.last()
    return df
    

In [377]:
# Test your function here.
class Criteria(Enum):
    ARTIST = 1,
    ALBUM = 2,
    GENRE = 3
MyAudioFilesOrganizer("./media/audio/*", Criteria.ARTIST)


ValueError: {'TIT2': TIT2(encoding=<Encoding.LATIN1: 0>, text=['Fire, Fire']), 'TPE1': TPE1(encoding=<Encoding.LATIN1: 0>, text=['M.I.A.']), 'TCOM': TCOM(encoding=<Encoding.LATIN1: 0>, text=['Maya Arulpragasam, Anthony Whiting']), 'TALB': TALB(encoding=<Encoding.LATIN1: 0>, text=['Arular']), 'TRCK': TRCK(encoding=<Encoding.LATIN1: 0>, text=['5/13']), 'TPOS': TPOS(encoding=<Encoding.LATIN1: 0>, text=['1/1']), 'TCON': TCON(encoding=<Encoding.LATIN1: 0>, text=['Hip Hop/Rap']), 'COMM:iTunPGAP:eng': COMM(encoding=<Encoding.LATIN1: 0>, lang='eng', desc='iTunPGAP', text=['0']), 'TENC': TENC(encoding=<Encoding.LATIN1: 0>, text=['iTunes v7.1']), 'COMM:iTunNORM:eng': COMM(encoding=<Encoding.LATIN1: 0>, lang='eng', desc='iTunNORM', text=[' 00000797 00000691 00006A5F 000074FA 0000D50F 0000F568 0000893B 0000891D 000072E5 000072E5']), 'COMM:iTunSMPB:eng': COMM(encoding=<Encoding.LATIN1: 0>, lang='eng', desc='iTunSMPB', text=[' 00000000 00000210 000008C4 00000000008C7A2C 00000000 003FA9F5 00000000 00

## Task 2.2 Audio mixer (25P):

In [320]:
# Write your function here.

# Cuts an audio file by delegating the cutting to FFMPEG.
def TwoAudioMixer(audioFile1: str, a1From: int, a1To: int, 
                  audioFile2: str, a2From: int, a2To: int, overlapDur: float, 
                  outputDir: str, outFilename: str) -> None:
    pass

In [ ]:
# Test your function here.

## Task 2.3 Concealing speakers ID by lowering/increasing the audio pitch (20P):

In [ ]:
# Write your function here.

# Generates a pandas DataFrame of audio metadata.
def VoicePitchChanger(audioFile: str, shift: float, outputDir: str, outFilename: str) -> None:
    pass

In [ ]:
# Test your function here.